In [ ]:
import pandas as pd
import json
import re
import numpy as np

In [196]:
# load the dataset
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_dataset.json', 'r')
#, encoding='utf-8'
data = json.load(json_file)

In [197]:
# Extract the variables of the first page
name = data[0]['name'].copy()
links = data[0]['link'].copy()
birth_year = data[0]['birth_year'].copy()
birth_place = data[0]['birth_place'].copy()
death_year = data[0]['death_year'].copy()
death_place = data[0]['death_place'].copy()

In [198]:
# Get the full links of the second pages before creating the dataframe
full_links = []
for link in links:
    full = 'http://www.ifar.org/' + link
    full_links.append(full)

In [199]:
# Check that the variables was created successfully
full_links[:5]

['http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=2030&published=1&inPrep=1&author=']

In [201]:
len(full_links)

2937

In [200]:
# Create the first dataset containing all of page 1
page_one = pd.DataFrame(list(zip(name, full_links, birth_year, birth_place, death_year, death_place)),
                          columns = ['names', 'links', 'birth_year', 'birth_place', 'death_year', 'death_place'])

In [211]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,"<td class=""g_center_birth_year"">1552</td>","<td class=""g_center_birth_place"">Germany</td>","<td class=""g_center_death_year"">1615</td>","<td class=""g_center_death_place"">Czech Republi..."
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1898</td>","<td class=""g_center_birth_place"">Finland</td>","<td class=""g_center_death_year"">1976</td>","<td class=""g_center_death_place"">Finland</td>"
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1836</td>","<td class=""g_center_birth_place"">Italy</td>","<td class=""g_center_death_year"">1868</td>","<td class=""g_center_death_place"">Italy</td>"
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,"<td class=""g_center_birth_year"">1971</td>","<td class=""g_center_birth_place"">Algeria</td>","<td class=""g_center_death_year""></td>","<td class=""g_center_death_place""></td>"
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,"<td class=""g_center_birth_year"">1889</td>","<td class=""g_center_birth_place"">Spain</td>","<td class=""g_center_death_year"">1957</td>","<td class=""g_center_death_place"">Spain</td>"


In [212]:
# Clean all of the variables in the first page and add them again to the dataset
regex = r'^(\d{4})'
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

clean_byear = []
clean_bplace = []
clean_dyear = []
clean_dplace = []

for by in page_one['birth_year']:
    f = remove_html_tags(by)
    clean_byear.append(f)
    
for bp in page_one['birth_place']:
    q = remove_html_tags(bp)
    clean_bplace.append(q)
    
for dy in page_one['death_year']:
    e = remove_html_tags(dy)
    clean_dyear.append(e)
    
for dp in page_one['death_place']:
    w = remove_html_tags(dp)
    clean_dplace.append(w)

page_one['birth_year'] = pd.Series(clean_byear)
page_one['birth_place'] = pd.Series(clean_bplace)
page_one['death_year'] = pd.Series(clean_dyear)
page_one['death_place'] = pd.Series(clean_dplace)

In [214]:
# Make sure that the variables were cleaned correctly
page_one.tail()

,names,links,birth_year,birth_place,death_year,death_place
2932,"Zuccaro, Taddeo",http://www.ifar.org/artist_name.php?nameid=275...,1529,Italy,1566,Italy
2933,"Zuloaga, Ignacio",http://www.ifar.org/artist_name.php?nameid=273...,1870,Spain,1945,Spain
2934,"Zumthor, Peter",http://www.ifar.org/artist_name.php?nameid=269...,1943,Switzerland,,
2935,"Zuñiga, Francisco",http://www.ifar.org/artist_name.php?nameid=109...,1912,Costa Rica,1998,Mexico
2936,"Zurbarán, Francisco de",http://www.ifar.org/artist_name.php?nameid=110...,1598,Spain,1664,Spain


In [215]:
# Make a copy of the rest of the dataset
second_part = data[1:].copy()

In [216]:
len(second_part)

7926

In [217]:
# Clean all of the variables for the second and third pages
artist_one = []
box_info = []
artist_two = []
column_names = []
content = []

for s in second_part:
    for key, val in s.items():
        if key == 'artist_one':
            artist_one.append(val)
        elif key == 'box_info':
            box_info.append(val)
        elif key == 'artist_two':
            artist_two.append(val)
        elif key == 'column_names':
            column_names.append(val)
        elif key == 'content':
            content.append(val)

In [221]:
len(artist_one), len(name)

(2937, 2937)

In [456]:
# Clean the box
box_new = []
for i in box_info:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    join_them = list(filter(None, tempora))
    box_new.append(join_them)

box_new[0], len(box_new)

(['Published Catalogues',
  'Hans von Aachen, 1552-1615.',
  'click to learn more',
  'Jacoby, Joachim W.',
  'Munich ; Berlin : Deutscher Kunstverlag, 2000.'],
 2937)

In [458]:
# Clean the new artist variable
artist_three = []
for i in artist_two:
    gen = str(i).strip('Artist: ')
    artist_three.append(gen)
    
artist_three[1], len(artist_three)

('Rho, Manlio', 4989)

In [457]:
# Clean the new columns
cols_news = []
for i in column_names:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    cols_news.append(tempora)
    
cols_news[1], len(cols_news)

(['Item Title:',
  'Author:',
  'ISBN:',
  'Imprint:',
  'Language:',
  'Size:',
  'Pages:',
  'Illustrations:',
  'Concordance:',
  'Bibliography:',
  'Index:',
  'Exhibition List:',
  'Chronology:',
  'Content Note:',
  'Individual Entries Contain:',
  'Public Note:'],
 4989)

In [459]:
new_content = []
for i in content:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    new_content.append(tempora)
    
new_content[1], len(new_content)

(['Rho : catalogo generale.',
  'Caramel, Luciano',
  '8843533339',
  'Milan : Electa, 1990.',
  'Italian',
  '29 cm',
  '316',
  'b&w, some color',
  'No',
  'Yes',
  'No',
  'Yes',
  'No',
  'The catalogue is based on the Rho Archives in Como. It is divided into three parts: the chronological catalogue of the paintings, followed by separate catalogues covering drawings for textile designs and decorative projects. \n\nEntries provide title, date (if attributed, date is determined by style, exhibition information, etc), dimensions, written documentation found on the recto or verso. The catalogue also cites current owners, auction sale information, provenance, exhibition and bibliographic information, technical notes, and comments.',
  'Provenance; Exhib. Hist; Bibliog.; Comments',
  'After the introduction, there is a catalogue raisonné, followed by sections dealing with textile designs, graphic designs, and projects for interiors, exhibition displays, and architecture. \n\nThe appendi

In [403]:
third_page = pd.DataFrame(list(zip(artist_two, column_names, content)), columns = ['name', 'col_names', 'content'])

In [410]:
third_page

list

In [230]:
# Second dataset
page_two = pd.DataFrame(list(zip(artist_one, box_info)), columns = ['name', 'box'])

In [232]:
page_two.head()

,names,box
0,"Aachen, Hans von","[\r\n , Published C..."
1,"Acht, René Charles","[\r\n , Published C..."
2,"Adam, Otto","[\r\n , Published C..."
3,"Adams, Clinton","[\r\n , Published C..."
4,"Accardi, Carla","[\r\n , Published C..."


In [406]:
len(page_two)

2937

In [299]:
for n in page_two.box:
    for su in page_two.box[n]:
        som = str(su).strip('\r\n').strip('nan').strip()
        page_two.box[n].append(som)
#    if temp == '':
#        temp = np.nan

KeyError: "None of [Index(['\\r\\n                            ', 'Published Catalogues',\n       '\\r\\n\\r\\n                ',\n       '\\r\\n\\r\\n                                                ',\n       '\\r\\n                                ',\n       'Hans von Aachen, 1552-1615.                                    ',\n       'click to learn more', '\\r\\n                            ',\n       '\\r\\n                                                                    Jacoby, Joachim W.                                                                    ',\n       '\\r\\n                                ',\n       '\\r\\n                                Munich ; Berlin : Deutscher Kunstverlag, 2000.',\n       '\\r\\n                                ',\n       '\\r\\n                            ',\n       '\\r\\n                            \\r\\n                        ',\n       '\\r\\n                                        ', ' ',\n       '\\r\\n                \\r\\n        '],\n  

In [233]:
tags = page_two['box'].apply(pd.Series)

In [234]:
tags = tags.rename(columns = lambda x : 'tag_' + str(x))

In [235]:
pd.set_option('display.max_columns', 500)
tags.head()

,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15,tag_16,tag_17,tag_18,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28,tag_29,tag_30,tag_31,tag_32,tag_33,tag_34,tag_35,tag_36,tag_37,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47,tag_48,tag_49,tag_50,tag_51,tag_52,tag_53,tag_54,tag_55,tag_56,tag_57,tag_58,tag_59,tag_60,tag_61,tag_62,tag_63,tag_64,tag_65,tag_66,tag_67,tag_68,tag_69,tag_70,tag_71,tag_72,tag_73,tag_74,tag_75,tag_76,tag_77,tag_78,tag_79,tag_80,tag_81,tag_82,tag_83,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99,tag_100,tag_101,tag_102,tag_103,tag_104,tag_105,tag_106,tag_107,tag_108,tag_109,tag_110,tag_111,tag_112,tag_113,tag_114,tag_115,tag_116,tag_117,tag_118,tag_119,tag_120,tag_121,tag_122,tag_123,tag_124,tag_125,tag_126,tag_127,tag_128,tag_129,tag_130,tag_131,tag_132,tag_133,tag_134,tag_135,tag_136,tag_137,tag_138,tag_139,tag_140,tag_141,tag_142,tag_143,tag_144,tag_145,tag_146,tag_147,tag_148,tag_149,tag_150,tag_151,tag_152,tag_153,tag_154,tag_155,tag_156,tag_157,tag_158,tag_159,tag_160,tag_161,tag_162,tag_163,tag_164,tag_165,tag_166,tag_167,tag_168,tag_169,tag_170,tag_171,tag_172,tag_173,tag_174,tag_175,tag_176,tag_177,tag_178,tag_179,tag_180,tag_181,tag_182,tag_183,tag_184,tag_185,tag_186,tag_187,tag_188,tag_189,tag_190,tag_191,tag_192,tag_193,tag_194,tag_195,tag_196,tag_197,tag_198,tag_199,tag_200,tag_201,tag_202,tag_203,tag_204,tag_205,tag_206,tag_207,tag_208,tag_209,tag_210,tag_211,tag_212,tag_213,tag_214,tag_215,tag_216,tag_217,tag_218,tag_219,tag_220,tag_221,tag_222,tag_223,tag_224,tag_225,tag_226,tag_227,tag_228,tag_229,tag_230,tag_231,tag_232,tag_233,tag_234,tag_235,tag_236,tag_237,tag_238,tag_239,tag_240,tag_241,tag_242,tag_243,tag_244,tag_245,tag_246,tag_247,tag_248,tag_249,...,tag_517,tag_518,tag_519,tag_520,tag_521,tag_522,tag_523,tag_524,tag_525,tag_526,tag_527,tag_528,tag_529,tag_530,tag_531,tag_532,tag_533,tag_534,tag_535,tag_536,tag_537,tag_538,tag_539,tag_540,tag_541,tag_542,tag_543,tag_544,tag_545,tag_546,tag_547,tag_548,tag_549,tag_550,tag_551,tag_552,tag_553,tag_554,tag_555,tag_556,tag_557,tag_558,tag_559,tag_560,tag_561,tag_562,tag_563,tag_564,tag_565,tag_566,tag_567,tag_568,tag_569,tag_570,tag_571,tag_572,tag_573,tag_574,tag_575,tag_576,tag_577,tag_578,tag_579,tag_580,tag_581,tag_582,tag_583,tag_584,tag_585,tag_586,tag_587,tag_588,tag_589,tag_590,tag_591,tag_592,tag_593,tag_594,tag_595,tag_596,tag_597,tag_598,tag_599,tag_600,tag_601,tag_602,tag_603,tag_604,tag_605,tag_606,tag_607,tag_608,tag_609,tag_610,tag_611,tag_612,tag_613,tag_614,tag_615,tag_616,tag_617,tag_618,tag_619,tag_620,tag_621,tag_622,tag_623,tag_624,tag_625,tag_626,tag_627,tag_628,tag_629,tag_630,tag_631,tag_632,tag_633,tag_634,tag_635,tag_636,tag_637,tag_638,tag_639,tag_640,tag_641,tag_642,tag_643,tag_644,tag_645,tag_646,tag_647,tag_648,tag_649,tag_650,tag_651,tag_652,tag_653,tag_654,tag_655,tag_656,tag_657,tag_658,tag_659,tag_660,tag_661,tag_662,tag_663,tag_664,tag_665,tag_666,tag_667,tag_668,tag_669,tag_670,tag_671,tag_672,tag_673,tag_674,tag_675,tag_676,tag_677,tag_678,tag_679,tag_680,tag_681,tag_682,tag_683,tag_684,tag_685,tag_686,tag_687,tag_688,tag_689,tag_690,tag_691,tag_692,tag_693,tag_694,tag_695,tag_696,tag_697,tag_698,tag_699,tag_700,tag_701,tag_702,tag_703,tag_704,tag_705,tag_706,tag_707,tag_708,tag_709,tag_710,tag_711,tag_712,tag_713,tag_714,tag_715,tag_716,tag_717,tag_718,tag_719,tag_720,tag_721,tag_722,tag_723,tag_724,tag_725,tag_726,tag_727,tag_728,tag_729,tag_730,tag_731,tag_732,tag_733,tag_734,tag_735,tag_736,tag_737,tag_738,tag_739,tag_740,tag_741,tag_742,tag_743,tag_744,tag_745,tag_746,tag_747,tag_748,tag_749,tag_750,tag_751,tag_752,tag_753,tag_754,tag_755,tag_756,tag_757,tag_758,tag_759,tag_760,tag_761,tag_762,tag_763,tag_764,tag_765,tag_766
0,\r\n,Published Catalogues,\r\n\r\n,\r\n\r\n ...,\r\n,"Hans von Aachen, 1552-1615. ...",click to learn m

In [327]:
tags.tag_555.unique()

array([nan, 'click to learn more'], dtype=object)

In [385]:
def clean_cols(series):
    import re
    clean = re.compile("\r\n* *")
    for i in series:
        temp = str(i)#.strip('nan').strip('click to learn more')
        re.sub(clean, '', temp)
        if temp == '':
            i = np.nan
    return series

In [388]:
copy_this = tags.tag_514.copy()

In [398]:
import re
clean = re.compile(r'\s*')
for i in copy_this:
    clean.findall(str(i))
    if i != '':
        i = np.nan
copy_this.unique()

array([nan, '\r\n                                    '], dtype=object)

In [386]:
print(clean_cols(tags.tag_13).unique())

['\r\n                            \r\n                        '
 'Sponsor: Steve Andreas                                '
 '\r\n                        '
 '\r\n                                        '
 'Sponsor: Robert Irwin Catalogue Raisonné LLC                                '
 'Sponsor: Dorotheum                                '
 'Sponsor: Estate of F. Luis Mora                                '
 'Sponsor: Fundación Zúñiga Laborde A.C.                                '
 'Sponsor: Fundación Azcona, Madrid                                '
 'Sponsor: The Fanzhi Foundation                                '
 'Sponsor: The Offices of Andrew Wyeth                                '
 'Sponsor: Alan Petersen                                '
 'Sponsor: Lunder Foundation                                '
 '\r\n\r\n                                        '
 'Sponsor: Anke Ariane Van Wagenberg-Ter Hoeven                                '
 'Sponsor: Andy Warhol Foundation for the Visual Arts, Inc.    

In [272]:
estri = []
for i in tags.tag_552:
    temp = str(i).strip('\r\n').strip('nan').strip()
    if temp == '':
        temp = np.nan
    estri.append(temp)

In [273]:
test_552 = pd.Series(estri)

In [294]:
test_552.isnull().count()

2937

In [295]:
q = 0
for i in test_552:
    if i == NaN:
        q += 1
print(q)

NameError: name 'NaN' is not defined

In [277]:
def strip_all_cols(series):
    temp = []
    for i in series:
        temp.append(str(i).strip('\r\n').strip())
        return temp
    
strip_all_cols(tags.tag_0)

temp[0:10]

TypeError: 'float' object is not subscriptable

In [190]:
tags.apply(str().strip('\r\n').strip())

AttributeError: 'DataFrame' object has no attribute ''

In [ ]:
cl_list = []
new = []

for i in range(len(second_pages.box)):
    for s in second_pages.box[i]:
        print(s)
        #si = s.strip('\r\n').strip()
        #new.append(si)
        
    #u = filter(None, i.strip('\r\n').strip())
    #cl_list.append(u)

In [ ]:
cl_list[1]

In [ ]:
new_clean = [list(filter(None, i)) for i in cl_list]
new_clean[:10]